In [1]:
import warnings # 警告模組
warnings.filterwarnings('ignore')

import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
import random

cat = 'all' 
# cat = 'all'： 全部
# cat = 'BKA'： 圖書
# cat = 'EBA'： 電子書
# cat = 'AVA'： 影音
# cat = 'DPA'： 百貨
# cat = 'MGA'： 雜誌
# cat = 'all/ovs/1'： 海外專館

url_main = "http://search.books.com.tw/search/query/key/%s/cat/%s"

url_page = "https://search.books.com.tw/search/query/cat/%s/sort/1/v/0/ms2/ms2_1/page/%s/key/%s"



headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36',
          }

def web_crawler(item_name, total_pages, books, prices):
    
    for page in range(1, total_pages + 1):
    
        url = url_page % (cat, str(page), item_name)
        
        flag = True
        
        # 寫 while 的原因是連線會常常失敗(博客來網站伺服器拒絕 client 端連線)，
        # 所以重新嘗試連線，直到成功連線下載到資料為止
        while(flag):
            
            res = requests.get(url, headers = headers)  # 加上 headers = headers
    
            print("\n第%02d頁的網頁內容大小 = %d 位元組" % (page, len(res.text)))  # 可取得網頁內容大小
        
            if len(res.text) > 0:
                
                flag = False
                
            else:
                
                print('** 如特定頁數爬不到資料時，會延長時間間隔約 50-60 秒之間，並以無限迴圈方式重新嘗試，直到抓取該頁完整資料為止 **')
                time.sleep(random.uniform(50, 60))

        #print(res.text)   # 使用 text 屬性將取得網頁的原始碼呈現出來
    
        soup = BeautifulSoup(res.text, 'html.parser')   # 將 html格式的字串傳進 BeautifulSoup
        
        items = soup.find_all('tbody', id = re.compile("itemlist_[A-Z0-9]\d{9}"))    # 撈出所有頁面上相關的 tbody 區塊
    
        print('本頁面共有', len(items), '筆資料！')
        print('** 每一頁爬取的時間間隔約 30~40 秒之間 **')

        for item in items:
    
            book = item.select("div.box_1 a img")[0]['alt']
    
            #print(book)
    
            #加到 pd.Series,(drop = True)用以重設索引值，從 0 開始
            books = books.append(pd.Series([book])).reset_index(drop = True)
    
            price = item.select("ul.list-nav.clearfix li strong")
    
            if len(price) == 1:  # 只有優惠價格
        
                # .string 取 tag<strong></strong> 中的文字內容 
                prices = prices.append(pd.Series([price[0].string])).reset_index(drop = True) 
        
            elif (len(price) == 2): #有打折數 +價格
        
                # .string 取 tag<strong></strong> 中的文字內容 
                prices = prices.append(pd.Series([price[1].string])).reset_index(drop = True)
        
            else:
        
                break  
            
        # 讓程式睡個幾秒(建議 30~60 秒或更久)再繼續爬取下一頁資料，避免頻繁抓取被台灣證券交易所封鎖 IP 拒絕存取
        time.sleep((random.uniform(30, 40))) 
        

    print('\n\n')    

    # 書籍與價格項目索引值由 1 開始
    books.index += 1

    prices.index += 1

    #print(books, '\n\n')

    #print(prices)

    #合併成 DataFrame
    df = pd.DataFrame({'書名':books, '價格': prices})

    return df

def search_books_com_tw():
    
    item_name = input('請輸入要搜尋的項目：')

    url = url_main % (item_name, cat)
    
    try:
        
        res = requests.get(url, headers = headers)  # 加上 header = headers
    
        #res.encoding = "utf-8"    # 此行必須要加，否則中文變成亂碼
    
        #res.raise_for_status()

    except Exception as err:            # err 是系統自訂的錯誤訊息

        print("博客來主網頁下載失敗:\n %s" % (err))

    else:

        #print("\n網頁下載成功：使用偽裝瀏覽器方式擷取網路資料成功！")

        #print("\n網頁內容大小 = %d 位元組" % (len(htmlfile.text)))  # 可取得網頁內容大小

        #print(res.text)   # 使用 text 屬性將取得網頁的原始碼呈現出來
    
        soup = BeautifulSoup(res.text, 'html.parser')   # 將 html格式的字串傳進 BeautifulSoup

        #print(soup)              # 直接印結果
        #print(soup.prettify())   # 如果想要呈現正常的 html 格式的話，可以多加 prettify()這個函式進來

        # 取得欲搜尋項目在網站上總頁數的內容
        page_info = soup.find('select', {'id':'page_select', 'onchange':'changePage(this)'})\
                    .find('option', {'selected': True}).text
    
        print(page_info)    
    
        total_pages = int( re.search(r'\d+', page_info).group(0) ) # 過濾出總頁數的數字內容
    
        return item_name, total_pages

if __name__ == "__main__":
    
    # 至博客來網站上執行搜尋任務
    item_name, total_pages = search_books_com_tw()
    
    # 準備好爬取書名的 Pandas Series 物件
    books = pd.Series([], dtype = object)

    # 準備好爬取價格的 Pandas Series 物件
    prices = pd.Series([], dtype = object)

    #
    df = web_crawler(item_name, total_pages, books, prices)
    
    

請輸入要搜尋的項目：ubuntu
共 20 頁

第01頁的網頁內容大小 = 74883 位元組
本頁面共有 24 筆資料！
** 每一頁爬取的時間間隔約 30~40 秒之間 **

第02頁的網頁內容大小 = 74158 位元組
本頁面共有 23 筆資料！
** 每一頁爬取的時間間隔約 30~40 秒之間 **

第03頁的網頁內容大小 = 77938 位元組
本頁面共有 23 筆資料！
** 每一頁爬取的時間間隔約 30~40 秒之間 **

第04頁的網頁內容大小 = 77121 位元組
本頁面共有 24 筆資料！
** 每一頁爬取的時間間隔約 30~40 秒之間 **

第05頁的網頁內容大小 = 68536 位元組
本頁面共有 24 筆資料！
** 每一頁爬取的時間間隔約 30~40 秒之間 **

第06頁的網頁內容大小 = 0 位元組
** 如特定頁數爬不到資料時，會延長時間間隔約 50-60 秒之間，並以無限迴圈方式重新嘗試，直到抓取該頁完整資料為止 **

第06頁的網頁內容大小 = 0 位元組
** 如特定頁數爬不到資料時，會延長時間間隔約 50-60 秒之間，並以無限迴圈方式重新嘗試，直到抓取該頁完整資料為止 **

第06頁的網頁內容大小 = 0 位元組
** 如特定頁數爬不到資料時，會延長時間間隔約 50-60 秒之間，並以無限迴圈方式重新嘗試，直到抓取該頁完整資料為止 **

第06頁的網頁內容大小 = 80184 位元組
本頁面共有 24 筆資料！
** 每一頁爬取的時間間隔約 30~40 秒之間 **

第07頁的網頁內容大小 = 81891 位元組
本頁面共有 22 筆資料！
** 每一頁爬取的時間間隔約 30~40 秒之間 **

第08頁的網頁內容大小 = 80667 位元組
本頁面共有 21 筆資料！
** 每一頁爬取的時間間隔約 30~40 秒之間 **

第09頁的網頁內容大小 = 78386 位元組
本頁面共有 24 筆資料！
** 每一頁爬取的時間間隔約 30~40 秒之間 **

第10頁的網頁內容大小 = 67623 位元組
本頁面共有 24 筆資料！
** 每一頁爬取的時間間隔約 30~40 秒之間 **

第11頁的網頁內容大小 = 72947 位元組
本頁面共有 20 筆資料！
** 每一頁爬取的時間間隔約 

In [2]:
df

,書名,價格
1,Ubuntu,810
2,Ubuntu,1575
3,Ubuntu 20管理入門與實作,474
4,Ubuntu 系統管理與架站實務(第三版),618
5,新時代 丙級電腦硬體裝修學術科研讀範本(含資訊類及技能檢定共同學科)使用Windows 10...,405
...,...,...
421,Walk With Us and Listen: Political Reconciliat...,1498
422,God’s Dream,630
423,Last Orders at Harrods: An African Tale,640
424,Guide to UNIX Using Linux,10898


In [3]:
df.style\
    .set_table_styles([ dict(selector = 'th',\
                             props = [('text-align', 'center'),\
                                        ('color', 'Green'), \
                                        ("font-size", "125%"),\
                                        ('background-color', '#f7f7f9')])])\
    .set_properties(**{'text-align': 'left', \
                        'background-color': 'black', \
                        'max-width': '650px',\
                        'color': 'lawngreen',\
                        'border-color': 'white'})

,書名,價格
1,Ubuntu,810
2,Ubuntu,1575
3,Ubuntu 20管理入門與實作,474
4,Ubuntu 系統管理與架站實務(第三版),618
5,新時代 丙級電腦硬體裝修學術科研讀範本(含資訊類及技能檢定共同學科)使用Windows 10 + Ubuntu 18附術科線上多媒體教學最新版(第十七版)(附MOSME行動學習一點通：學科.影音.診斷),405
6,Ubuntu19完全自學手冊：桌面、系統與網路應用全攻略,522
7,與Ubuntu共舞：中文環境調校x雲端共享x Libreoffice x 架站 x dropbox自己架(隨書附贈教學影片與Ububntu安裝光碟),379
8,Ubuntu 系統管理與架站實務(第2版),618
9,Linux進化特區：Ubuntu 13.04 從入門到精通,522
10,Ubuntu Linux網管手冊：網路服務、資通安全一手搞定,332
